<a href="https://colab.research.google.com/github/sbagency/AI-agents-hacks/blob/main/MTSAIR_Cotype_Nano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.0/87.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.3/210.3 kB

In [2]:
!pip install triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.3 MB/s eta 0:00:00


In [5]:
!nohup python3 -m vllm.entrypoints.openai.api_server --dtype=half --model MTSAIR/Cotype-Nano --port 8000  > vllm.out 2>&1 &

In [6]:
!ps -ax | grep vllm

   5711 ?        Sl     0:08 python3 -m vllm.entrypoints.openai.api_server --dtype=half --model MTSA
   5772 ?        Rl     0:06 /usr/bin/python3 -m vllm.model_executor.models.registry
   5804 ?        S      0:00 /bin/bash -c ps -ax | grep vllm
   5806 ?        S      0:00 grep vllm


In [9]:
!cat vllm.out

2024-11-26 12:08:26.536951: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-26 12:08:26.563134: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-26 12:08:26.575008: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-26 12:08:26.598062: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-26 12:08:28.245228: W tensorflow/comp

In [15]:
import openai
import pandas as pd
from tqdm import tqdm


model = 'MTSAIR/Cotype-Nano'

client = openai.OpenAI(
    base_url = 'http://localhost:8000/v1',
    api_key='xxx', # required, but unused
)


# Possible system prompt:
# {"role": "system", "content": "Ты — ИИ-помощник. Тебе дано задание: необходимо сгенерировать подробный и развернутый ответ."},
#    messages=[
#          {"role": "user", "content": "Как мне обучить модель meta-llama/Llama-3.2-1B с помощью библиотеки transformers?"}
#        ]
#    )

temperature=0.4 # 0.0 is also allowed
frequency_penalty=0.0
max_tokens=2048
top_p=0.8 # 0.1 is also allowed


#import time
#from openai import OpenAI
#from google.colab import userdata


def llm(messages):
  print(f"{model},temperature={temperature},top_p={top_p},max_tokens={max_tokens}")
  completion = client.chat.completions.create(
  model=model,
  messages=messages,
  temperature=temperature,
  frequency_penalty=frequency_penalty,
  top_p=top_p,
  max_tokens=max_tokens,
  stream=False
  )
  if not hasattr(completion, 'choices'):
    print(completion)
    raise Exception(completion)

  return completion.choices[0].message.content


def llmp(prompt):
  messages=[{"role": "user","content": prompt}]
  return llm(messages)

In [16]:
resp=llmp("Расскажи анекдот про лошадь")
print(resp)

MTSAIR/Cotype-Nano,temperature=0.4,top_p=0.8,max_tokens=2048
Конечно, вот один из анекдотов про лошадь:

---

Однажды лошадь спрашивает другую лошадь:

— Ты не боишься, что я могу тебя убить?

Вторая лошадь отвечает:

— Нет, боюсь, что ты не сможешь меня убить.

---

Надеюсь, этот анекдот вам понравился!
